## Named Tuples Apps

### Returning Multiple

In [215]:
from random import randint, random

In [217]:
from collections import namedtuple

In [235]:
def random_color():
    red = randint(0,255)
    blue = randint(0,255)
    green = randint(0,255)
    alpha = round(random(),2)
    return red, green, blue, alpha

In [222]:
color = random_color()

In [223]:
color

(132, 222, 77, 0.58)

Prostě máme generátor hodnot pro barvu, nic víc.

In [224]:
red, blue, green, alpha = color

In [226]:
red, blue, green, alpha

(132, 222, 77, 0.58)

Můžu takto unpacknout hodnoty. A teď pomocí namedtuple.

In [227]:
Color = namedtuple("Color", "red green blue alpha")

In [234]:
def random_color():
    red = randint(0,255)
    blue = randint(0,255)
    green = randint(0,255)
    alpha = round(random(),2)
    return Color(red, green, blue, alpha)
    #v podstatě moje funkce returní NamedTuple 
    #protože color je named tuple a argumenty sedí

In [232]:
color = random_color()

In [233]:
color

Color(red=74, green=231, blue=198, alpha=0.74)

Zajímavé, takto si to propojíme a tvoříme named tuple v pdostatě pomocí funkce.

Obecně vhodné mít například pycharm pro našeptávání. VS code je +- asi mizernější stav.

### Alternative to Dictionaries

In [236]:
data_dict = dict(key1=100, key2=200, key3=300)

In [237]:
data_dict["key1"]

100

Pokud nechceme modifikovat dictionary, někdy je vhodnější použít named tuple.

In [238]:
from collections import namedtuple

In [239]:
Data = namedtuple("Data", data_dict.keys())

In [240]:
Data._fields

('key1', 'key2', 'key3')

Vytvořili jsme namedtuple s hodnotami z dicitonary.

In [252]:
d1 = Data(*data_dict.values())
#a nyní vytvářím objetk z naší named tuple classy - a již ne pomocí klíčů ale 
#pomocí hodnot :)
#unpoacking protože rozbaluji list - zatímco při tvorbě si poradí se zápisem "x y z"

In [243]:
d1 #tadááá

Data(key1=100, key2=200, key3=300)

In [249]:
data_dict.values()
#můžu vidět že navrací list - proto unpacking viz výše

dict_values([100, 200, 300])

In [253]:
d2 = Data(key1=10, key3=30, key2=20)

In [254]:
d2

Data(key1=10, key2=20, key3=30)

In [255]:
Data._fields

('key1', 'key2', 'key3')

Obecně jde o problém, že se nám může pomíchat pořadí klíčů.

In [257]:
d2 = Data(**data_dict)

In [258]:
d2

Data(key1=100, key2=200, key3=300)

Ale tohle je badass, prostě jej zavedu s dictionary a provedu ho takto snadno do named tuple. Předtím si vytvořím samozřejmě šablonu.

In [262]:
data_list = [
    { "key1": 1, "key2": 2},
    { "key1": 3, "key2": 4},
    { "key1": 5, "key2": 6, "key3" : 7},
    { "key2": 100}
]

Začínáme s listem dictionaries.

In [263]:
keys = set() #budeme získávat unikátní hodnoty

In [264]:
for d in data_list:
    for key in d.keys():
        keys.add(key)
     #iterujeme data list a dicitonary a vkládáme   
print(keys)

{'key3', 'key2', 'key1'}


In [265]:
keys = {key for dict_ in data_list for key in dict_.keys()}

In [268]:
keys #stejné jako nahoře jen pomocí List Comprehension - stále zamotané

{'key1', 'key2', 'key3'}

In [270]:
Struct = namedtuple("Struct", sorted(keys)) #chci je mít seřazené jasně
#prostě zaládáme namedtuple

In [273]:
Struct._fields

('key1', 'key2', 'key3')

Problém je, že ne všechny dictionary mají 3 klíče, 3 jsou maximum.

In [275]:
Struct.__new__.__defaults__ = (None,) * len(Struct._fields)
#prostě dáváme podle délky toho kolik máme polí - defaultně hodnotu None

In [277]:
tuple_list = []
for dict_ in data_list:
    #takže budeme znovu iterovat skrze náš supr list s dictionary
    tuple_list.append(Struct(**dict_))
print(tuple_list)

[Struct(key1=1, key2=2, key3=None), Struct(key1=3, key2=4, key3=None), Struct(key1=5, key2=6, key3=7), Struct(key1=None, key2=100, key3=None)]


Tadááá :D

V podstatě řádky předtím nenavazují na finální kod, pouze ukazují postup jak jsme se k tomu dostali. Nyní to přepíšeme do jedné supr funkce.

In [286]:
def tuplify_dicts(dicts):
    """obdrž sequncy"""
    keys = {key for dict_ in dicts for key in dict_.keys()}
    #vydoluje unikátní klíče - dáváme do setu - set comprehension!
    #jinak syntax obecně doluje všechny klíče ze všech hodnot co najde
    #takto vytvoří univerzální unikátní seznam
    Struct = namedtuple("Struct", sorted(keys)) #vytvoříme classu z named tuple
    #která se jmenuje Struct a hodnoty classy jsou právě keys..
    Struct.__new__.__defaults__ = (None,)* len(Struct._fields)
    #nastavíme defaultní hodnotu pro classu struct - podle délky jeho polí ..
    #v podstatě - podle toho kolik má klíčů - tolik hodnot bude mít defaultních
    return [Struct(**dict_) for dict_ in dicts]
    #unpackuju všechny dictionary zvlášt do namedtuple classy :) ..
    #z listu dictionary unpackini dictionary - a udělej z něj named tuple classu

In [287]:
tuplify_dicts(data_list)

[Struct(key1=1, key2=2, key3=None),
 Struct(key1=3, key2=4, key3=None),
 Struct(key1=5, key2=6, key3=7),
 Struct(key1=None, key2=100, key3=None)]

In [297]:
[key for dict_ in data_list for key in dict_.keys()]

['key1', 'key2', 'key1', 'key2', 'key1', 'key2', 'key3', 'key2']

Pokud zjistím, že píšu classy, které jsou neměnné a disponují leda tak __init__ a pár parametry. Tak píši Named tuples :)